In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [132]:
DATA_PATH = '/home/capstone22/WildFIrePrediction/Isaac_data'

In [133]:
features = [
 'Elec_Dist',
 'aet__maximum_Normal',
 'aet__mean_Normal',
 'aet__minimum_Normal',
 'cwd__maximum_Normal',
 'cwd__mean_Normal',
 'cwd__minimum_Normal',
 'aet__mean_ThreeYear_Dev',
 'Mean_Housing_Dens_25km',
 'value']

In [134]:
print(len(features))

10


# Dataset preprocessing

In [135]:
class IsaacDS(Dataset):
    def __init__(self, split, features, data_dir):
        self.split = split
        self.features = features
        self.data_dir = data_dir
        self.df = self.getdf()
      

    def getdf(self):
        path = os.path.join(self.data_dir, f'isaac_{self.split}.csv')
        df = pd.read_csv(path)
        df = df[self.features]
        
        return df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        X = self.df.drop(columns=['value']).iloc[idx].values
        y = self.df['value'].iloc[idx]
        return torch.from_numpy(X).float(), torch.from_numpy(np.array([y])).float()

In [ ]:
# class IsaacDS(Dataset):
#     def __init__(self, split, features, data_dir):
#         self.split = split
#         self.features = features
#         self.data_dir = data_dir
#         self.df = self.getdf()
      

#     def getdf(self):
#         path = os.path.join(self.data_dir, f'isaac_{self.split}.csv')
#         df = pd.read_csv(path)
#         df = df[self.features]
        
#         return df
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, idx):
#         X = self.df.drop(columns=['value']).iloc[idx].values
#         y = self.df['value'].iloc[idx]
#         return torch.from_numpy(X).float(), torch.from_numpy(np.array([y])).float()

In [136]:
ds_train = IsaacDS('train', features, DATA_PATH)
ds_valid = IsaacDS('valid', features, DATA_PATH)
ds_test = IsaacDS('test', features, DATA_PATH)

In [106]:
print(ds_train[0])

(tensor([10.0000, 69.2570, 23.5285,  1.9532, 96.3592, 52.9545, 19.4323,  3.1711,
        26.7362]), tensor([0.]))


In [137]:
ds_train.df

,Elec_Dist,aet__maximum_Normal,aet__mean_Normal,aet__minimum_Normal,cwd__maximum_Normal,cwd__mean_Normal,cwd__minimum_Normal,aet__mean_ThreeYear_Dev,Mean_Housing_Dens_25km,value
0,10.000000,69.257042,23.528457,1.953188,96.359229,52.954543,19.432312,3.171076,26.736217,0.0
1,6.324555,68.618766,26.252643,3.016567,166.582232,84.301442,21.105000,0.850137,3.738432,0.0
2,4.242641,41.253983,14.183735,0.572433,195.247349,101.775402,21.919567,0.916099,28.166508,0.0
3,4.000000,74.506400,20.550272,0.577850,181.168049,87.073095,24.463767,0.269284,14.826780,0.0
4,10.816654,95.391396,39.021024,3.490667,89.660375,50.771127,18.809500,0.955209,6.530105,0.0
...,...,...,...,...,...,...,...,...,...,...
99995,0.000000,41.424107,17.575733,3.042240,151.668040,95.345716,45.695733,0.923710,139.827637,0.0
99996,1.000000,41.942494,13.927058,0.796507,158.723654,75.709296,8.392253,0.177176,8.381964,0.0
99997,2.236068,51.403616,19.017093,1.888033,163.540899,93.918676,44.238700,0.099018,51.117859,0.0
99998,4.242641,18.071150,2.923032,0.000000,202.093332,110.138842,31.207217,0.335885,5.411052,0.0


# Model building

In [138]:
class Model(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Model, self).__init__()
        # Linear function
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim) 
        self.sigmoid = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out

In [139]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [144]:
input_dim = len(features) - 1
hidden_dim = 10
output_dim = 1

model = Model(input_dim, hidden_dim, output_dim)
model.to(device)

Model(
  (fc1): Linear(in_features=9, out_features=10, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)

In [145]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

criterion = torch.nn.CrossEntropyLoss()

# Model Training

In [147]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch}-------------------------------------')
    for i, (X, y) in enumerate(ds_train):
        
        X = X.to(device).view(1,-1)
        y = y.to(device).view(1,-1)

        outputs = model(X)
        loss = criterion(outputs, y)
        optimizer.zero_grad()

        loss.backward()

        # Updating parameters
        optimizer.step()
    print('completed', epoch)
    iter = 0
    for i in range(5):
        for X, y in ds_valid:
            prediction = model(X.to(device))
            print(prediction.item())



Epoch 0-------------------------------------
completed 0
0.6028443574905396
0.8493074774742126
0.8833439946174622
0.5968964099884033
0.8837002515792847
0.8833059072494507
0.7264988422393799
0.597646951675415
0.8834435939788818
0.8836719393730164
0.8842880129814148
0.6052072644233704
0.883510410785675
0.8833063244819641
0.5967304706573486
0.5967308282852173
0.8832981586456299
0.8833546042442322
0.8832981586456299
0.5967180728912354
0.8833303451538086
0.8832980394363403
0.8833112120628357
0.8832981586456299
0.8833064436912537
0.8833693861961365
0.6146736145019531
0.4193479120731354
0.8832982778549194
0.6026086211204529
0.5967170596122742
0.8833478093147278
0.8833044171333313
0.6633052229881287
0.8832982182502747
0.4470636546611786
0.8832982182502747
0.5989184379577637
0.5872350931167603
0.8833377361297607
0.5484152436256409
0.8833305239677429
0.8834605813026428
0.883328378200531
0.6467382907867432
0.8835700154304504
0.5927035808563232
0.8833182454109192
0.5966997146606445
0.5634694695472

KeyboardInterrupt: 